In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **import the dataset !!before!! install autotrain**

In [2]:
from datasets import load_dataset
dataset = load_dataset("camel-ai/math")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
import os
!pip install -U autotrain-advanced 
!autotrain setup

  Using cached protobuf-4.23.4-cp37-abi3-manylinux2014_x86_64.whl.metadata (540 bytes)
  Using cached cryptography-42.0.5-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached pydantic_core-2.18.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.5 kB)
Using cached cryptography-42.0.5-cp39-abi3-manylinux_2_28_x86_64.whl (4.6 MB)
Using cached protobuf-4.23.4-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Using cached pydantic_core-2.18.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.1
    Uninstalling pydantic_core-2.18.1:
      Successfully uninstalled pydantic_core-2.18.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobu

In [5]:
data = dataset["train"]
data

Dataset({
    features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
    num_rows: 50000
})

v1 : 0;3500
v1.1 : 3500;7000

In [6]:
import pandas as pd
data = {"text":[]}

msg1 = dataset["train"]["message_1"]
msg2 = dataset["train"]["message_2"]
print(len(msg1))
for i in range(3500, 7000):
    user = "<|user|>"+ msg1[i] +"<|end|>\n"
    phi = "<|assistant|>"+ msg2[i] +"<|end|>"
    prompt = user+phi
    data["text"].append(prompt)
    
data = pd.DataFrame.from_dict(data)
print(data)
os.mkdir("/kaggle/working/data")
data.to_csv('data/dataset.csv', index=False, escapechar='\\')  

50000
                                                   text
0     <|user|>Sure, here's a specific question for t...
1     <|user|>A cone has a radius of 7 cm and a slan...
2     <|user|>A cone has a radius of 5 cm and a slan...
3     <|user|>A cone has a radius of 8 cm and a slan...
4     <|user|>What is the surface area of a cone wit...
...                                                 ...
3495  <|user|>Find the maximum and minimum values of...
3496  <|user|>A water tank in the shape of a rectang...
3497  <|user|>Find the maximum and minimum values of...
3498  <|user|>A ball is thrown upward from the top o...
3499  <|user|>Find the maximum and minimum values of...

[3500 rows x 1 columns]


In [7]:
pd.read_csv('data/dataset.csv')

,text
0,"<|user|>Sure, here's a specific question for t..."
1,<|user|>A cone has a radius of 7 cm and a slan...
2,<|user|>A cone has a radius of 5 cm and a slan...
3,<|user|>A cone has a radius of 8 cm and a slan...
4,<|user|>What is the surface area of a cone wit...
...,...
3495,<|user|>Find the maximum and minimum values of...
3496,<|user|>A water tank in the shape of a rectang...
3497,<|user|>Find the maximum and minimum values of...
3498,<|user|>A ball is thrown upward from the top o...


In [8]:
#@markdown ---
#@markdown #### Project Config
#@markdown Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.
project_name = 'competition-math-phinetune-v1-1' # @param {type:"string"}
model_name = "styalai/competition-math-phinetune-v1" #'microsoft/Phi-3-mini-4k-instruct' # @param {type:"string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = True # @param ["False", "True"] {type:"raw"}
hf_token = "" #@param {type:"string"}
#repo_id = "styalai/phine_tuning_1" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 3e-4 # @param {type:"number"}
num_epochs = 1 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
block_size = 1024 # @param {type:"number"}
trainer = "sft" # @param ["default", "sft"] {type:"raw"}
warmup_ratio = 0.1 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
gradient_accumulation = 4 # @param {type:"number"}
mixed_precision = "fp16" # @param ["fp16", "bf16", "none"] {type:"raw"}
peft = True # @param ["False", "True"] {type:"raw"}
quantization = "int4" # @param ["int4", "int8", "none"] {type:"raw"}
lora_r = 16 #@param {type:"number"}
lora_alpha = 32 #@param {type:"number"}
lora_dropout = 0.05 #@param {type:"number"}

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
#os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["MIXED_PRECISION"] = str(mixed_precision)
os.environ["PEFT"] = str(peft)
os.environ["QUANTIZATION"] = str(quantization)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)

**num sample / block size must be > 50**

In [ ]:
!autotrain llm -help

In [9]:
!autotrain llm \
--train \
--username "styalai" \
--merge-adapter \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--quantization ${QUANTIZATION} \
--mixed-precision ${MIXED_PRECISION} \
$( [[ "$PEFT" == "True" ]] && echo "--peft" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN}" )

INFO     | 2024-05-02 15:52:13 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2024-05-02 15:52:13 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: backend, deploy, inference, train, version, func
INFO     | 2024-05-02 15:52:13 | autotrain.backend:create:300 - Starting local training...
INFO     | 2024-05-02 15:52:13 | autotrain.commands:launch_command:349 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'competition-math-phinetune-v1-1/training_params.json']
INFO     | 2024-05-02 15:52:13 | autotrain.commands:launch_command:350 - {'model': 'styalai/competition-math-phinetune-v1', 'project_name': 'competition-math-phinetune-v1-1', 'data_path': 'data/', 'train_split': 'train', 'valid_split': None, 'add_eos_token': False, 'block_size': 1024, 'model_max_length': 1024, 'padding': None, 'trainer': 'default', 'use_flash_attention_2': False, 'log': 'n